# Tutorial on how to use mescal
*mescal* is a [Brightway](https://docs.brightway.dev/en/latest/)-powered Python package that helps you to integrate Life-Cycle Assessment (LCA) in your energy system model

## Set up

In [1]:
# Import the required libraries
from mescal import *
import pandas as pd
import bw2data as bd

In [2]:
ecoinvent_version = '3.9.1' # choose the ecoinvent version you wish to use
esm_location = 'CA-QC' # choose the version of energyscope for which you want to generate metrics
esm_year = 2050 # choose the year of the energyscope snapshot model or transition pathway time-step
regionalized_database = True # set to True if you want to use your regionalized version of ecoinvent
regionalize_foregrounds = True # set to True if you want to regionalize the foreground inventories
premise_iam = 'image' # choose the IAM to which the premise database is linked
premise_ssp_rcp = 'SSP2-Base' # choose the SSP/RCP scenario to which the premise database is linked

In [3]:
# Set the name of your main LCI database (e.g., ecoinvent or premise database) here:
name_premise_db = f"ecoinvent_cutoff_{ecoinvent_version}_{premise_iam}_{premise_ssp_rcp}_{esm_year}+truck_carculator"
if regionalized_database:
    name_premise_db += ' regionalized'
name_premise_with_CPC_db = name_premise_db + '_with_CPC'

In [4]:
# Set the name of the new database with the ESM results
new_db_name = f'energyscope_{esm_location}_{esm_year}'

In [5]:
# Set the list of LCIA methods for which you want indicators (they must be registered in your brightway project)
if regionalized_database:
    lcia_methods=[
        'IMPACT World+ Midpoint 2.0.1_regionalized', 'IMPACT World+ Damage 2.0.1_regionalized', 'IMPACT World+ Footprint 2.0.1_regionalized', # IW+ regionalized
    ]
else:
    lcia_methods=[
        'IMPACT World+ Midpoint 2.0.1', 'IMPACT World+ Damage 2.0.1', 'IMPACT World+ Footprint 2.0.1', # IW+
        'PB LCIA' # Planetary Boundaries
    ]

**Note**: you can download IMPACT World+ methods in your brightway project following this [notebook](https://github.com/matthieu-str/mescal/blob/master/dev/download_impact_world_plus.ipynb). Regionalized versions of ecoinvent and IMPACT World+ methods can be downloaded from this [repository](https://github.com/CIRAIG/Regioinvent).

In [6]:
# Set up your Brightway project
bd.projects.set_current(f'ecoinvent{ecoinvent_version}') # put the name of your brightway project here

## Your data
For mescal to understand the structure of your energy system model, you need to provide it with a set of dataframes.

### Mandatory dataframes

In [7]:
mapping = pd.read_csv(f'../dev/energyscope_data/{esm_location}/mapping_{ecoinvent_version}.csv')
unit_conversion = pd.read_csv(f'../dev/energyscope_data/{esm_location}/unit_conversion_{ecoinvent_version}.csv')
mapping_esm_flows_to_CPC = pd.read_csv(f'../dev/energyscope_data/{esm_location}/mapping_esm_flows_to_CPC.csv')
model = pd.read_csv(f'../dev/energyscope_data/{esm_location}/model.csv')

A mapping between the energy technologies and resources of the energy system model, and Life-Cycle Inventories datasets (LCI) from an LCI database (e.g., ecoinvent). The mapping should be provided in a dataframe with the following columns:
- `Name`: the name of the energy technology or resource in the energy model 
- `Type`: the type of the energy technology or resource (i.e., 'Construction', 'Operation', or 'Resource')  
- `Product`: the name of the product of the energy technology or resource in the LCI database
- `Activity`: the name of the activity of the energy technology or resource in the LCI database
- `Location`: the name of the region of the energy technology or resource in the LCI database
- `Unit`: (optional) the physical unit of the energy technology or resource in the LCI database
- `Database`: the name of the database in your brightway project

If you wish to change the version of ecoinvent used in your mapping file, you can follow this [notebook](https://github.com/matthieu-str/mescal/blob/master/dev/mapping_ecoinvent_version.ipynb). 

In [8]:
mapping.head()

,Name,Type,Product,Activity,Location,Database
0,AFC,Construction,"fuel cell system, 1 kWe, proton exchange membr...","fuel cell system assembly, 1 kWe, proton excha...",GLO,ecoinvent_cutoff_3.9.1_image_SSP2-Base_2020
1,AFC,Operation,"electricity, from residential heating system","electricity, residential, by conversion of hyd...",CH,ecoinvent_cutoff_3.9.1_image_SSP2-Base_2020
2,ALKALINE_ELECTROLYSIS,Operation,"hydrogen, gaseous, 20 bar","hydrogen production, gaseous, 20 bar, from AEC...",CH,ecoinvent_cutoff_3.9.1_image_SSP2-Base_2020
3,ALKALINE_ELECTROLYSIS_PLANT,Construction,"electrolyzer, 1MWe, AEC, Balance of Plant","electrolyzer production, 1MWe, AEC, Balance of...",RER,ecoinvent_cutoff_3.9.1_image_SSP2-Base_2020
4,ALKALINE_ELECTROLYSIS_PLANT_DECOM,Construction,"used fuel cell balance of plant, 1MWe, AEC","treatment of fuel cell balance of plant, 1MWe,...",RER,ecoinvent_cutoff_3.9.1_image_SSP2-Base_2020


A set of unit conversion factors between the energy system model and the LCI database. The conversion factors should be provided in a dataframe with the following columns:
- `Name`: the name of the energy technology or resource in the energy model 
- `Type`: the type of the energy technology or resource (i.e., 'Construction', 'Operation', 'Resource', or 'Other'). Other types can be added to fit your needs. The 'Other' category is meant for the unit conversion that are not specific to a technology or resource, but rather a generic type of product, e.g., conversion from kilogram to kilowatt hour for natural gas.
- `Value`: the numerical value of the conversion factor that will multiply the impact scores. It actually denotes the conversion from Impact / LCA unit to Impact / ESM unit. 
- `ESM`: the unit of the energy technology or resource in the ESM (`From` in v1.0.1)
- `LCA`: the target unit of the energy technology or resource in the LCA database (`To` in v1.0.1)

The `LCA` and `ESM` columns should respect the ecoinvent naming convention. You may use the `ecoinvent_unit_convention` function to convert the unit naming convention you've used to the one of ecoinvent.

In [9]:
unit_conversion.head()

,Name,Type,Value,LCA,ESM
0,ACETIC_ACID,Resource,0.247423,kilogram,kilowatt hour
1,ACETONE,Resource,0.121655,kilogram,kilowatt hour
2,AFC,Construction,1.000000,unit,kilowatt
3,AFC,Operation,1.000000,kilowatt hour,kilowatt hour
4,ALKALINE_ELECTROLYSIS,Construction,0.001556,unit,kilowatt


A mapping between the energy model flows and CPC categories. The list of products should at least encompass the list of technologies without a CPC category within your mapping and their foreground inventory. The mapping should be provided in a dataframe with the following columns:
- `Product`: the name of the product in the LCI database
- `Description`: (optional) the description of the product
- `CPC`: the list of names of corresponding CPC categories

In [10]:
mapping_esm_flows_to_CPC.head()

,Flow,Description,CPC
0,BENZENE,Benzene,"['33100: Coke and semi-coke of coal, of lignit..."
1,BIO_DIESEL,Bio-diesel,"['35491: Biodiesel', '33370: Fuel oils n.e.c.'..."
2,CO2_A,Carbon dioxide (concentrated emissions),['34210b: Carbon dioxide and monoxide']
3,CO2_C,Carbon dioxide (captured),['34210b: Carbon dioxide and monoxide']
4,CO2_CS,Carbon dioxide (captured & stored),['34210b: Carbon dioxide and monoxide']


The input and output flows if energy technologies. For a given technology, the inputs and outputs should be given with the same unit. Also, the main output flow should have 1 as an amount, i.e., all other flows as scaled with respect to the main output. It should be provided in a dataframe with the following columns:
- `Name`: the name of the energy technology in the energy model
- `Flow`: the name of the input or output flow
- `Amount`: the numerical value of the flow (negative if input, positive if output)  

In [11]:
model.head()

,Name,Flow,Amount
0,DIESEL_S,DIESEL_S,1.0
1,GASOLINE_S,GASOLINE_S,1.0
2,ELEC_S,ELEC_S,1.0
3,NG_S,NG_S,1.0
4,H2_S,H2_S,1.0


### Optional dataframes

In [12]:
technology_compositions = pd.read_csv(f'../dev/energyscope_data/{esm_location}/technology_compositions.csv')
technology_specifics = pd.read_csv(f'../dev/energyscope_data/{esm_location}/technology_specifics.csv') 
lifetime = pd.read_csv(f'../dev/energyscope_data/{esm_location}/lifetime.csv')
efficiency = pd.read_csv(f'../dev/energyscope_data/{esm_location}/efficiency.csv')
mapping_product_to_CPC = pd.read_csv('../mescal/data/mapping_product_to_CPC.csv')
impact_abbrev = pd.read_csv('../dev/lcia/impact_abbrev.csv')
technologies_to_remove_from_layers = pd.read_csv(f'../dev/energyscope_data/{esm_location}/technologies_to_remove_from_layers.csv')
new_end_use_types = pd.read_csv(f'../dev/energyscope_data/{esm_location}/new_end_use_types.csv')
premise_changes = pd.read_csv('../dev/data/premise_change_report.csv')
results_from_esm = pd.read_csv(f'../dev/energyscope_data/{esm_location}/results_ES.csv')

A set of composition of technologies, i.e., if one technology or resource in the energy model should be represented by a combination of LCI datasets. The composition should be provided in a dataframe with the following columns:
- `Name`: the name of the main energy technology or resource in the energy model 
- `Components`: the list of names of subcomponents

In [13]:
technology_compositions.head()

,Name,Components
0,ALKALINE_ELECTROLYSIS,"['ALKALINE_ELECTROLYSIS_STACK', 'ALKALINE_ELEC..."
1,AN_DIG_SI,"['AN_DIG_SI_PLANT', 'AN_DIG_SI_COGEN']"
2,ATR,"['ATR_PLANT', 'ATR_TANK']"
3,ATR_CCS,"['ATR_CCS_PLANT', 'ATR_CCS_TANK']"
4,BIOGAS_ATR,"['BIOGAS_ATR_PLANT', 'BIOGAS_ATR_TANK']"


A set of technologies with specific requirements. For instance, this stands for energy technologies without a construction phase, mobility technologies (if mismatch fuel in the LCI dataset), bioprocesses (if mismatch fuel in the LCI dataset). The requirements should be provided in a dataframe with the following columns:
- `Name`: the name of the energy technology in the energy model
- `Specifics`: the list of requirements  
- `Amount`: the numerical value of the requirement (if relevant)

In [14]:
technology_specifics.head()

,Name,Specifics,Amount
0,CO2_TO_DIESEL,Background search,2.0
1,CO2_TO_JETFUELS,Background search,0.0
2,CROPS_TO_JETFUELS,Background search,4.0
3,FT,Background search,3.0
4,GASIFICATION_SNG,Background search,2.0


Energy technologies lifetimes in the ESM and the LCI database. The lifetimes should be provided in a dataframe with the following columns:
- `Name`: the name of the energy technology in the energy model
- `ESM`: the numerical value of the lifetime in the energy system model (if relevant)
- `LCA`: the numerical value of the lifetime in the LCI database (if relevant)

In [15]:
lifetime.head()

,Name,ESM,LCA
0,AFC,35.0,2.686006
1,AL_MAKING,25.0,50.000000
2,AL_MAKING_HR,25.0,50.000000
3,ALKALINE_ELECTROLYSIS,10.0,NaN
4,ALKALINE_ELECTROLYSIS_PLANT,NaN,20.000000


If you want to account for the possible efficiency differences between the ESM and the LCI datasets, you can provide a set of (technologies, flow) couples for which efficiency differences will be corrected by the scaling the elementary flows. Taking the example of the diesel car, the couple should be something like ('CAR_DIESEL', 'DIESEL'), the flow being the fuel of the technology responsible for direct emissions. Relevant biosphere flows will be the ratio between the LCA and ESM efficiencies. The (technology, flow) couples should be provided in a dataframe with the following columns:
- `Name`: the name of the energy technology in the ESM
- `Flow`: the name of the flow in the ESM

In [16]:
efficiency.head()

,Name,Flow
0,AN_DIG,WET_BIOMASS
1,AN_DIG_SI,WET_BIOMASS
2,ATR_CCS,NG_HP
3,ATR,NG_HP
4,BIOGAS_ATR_CCS,WOOD


In case you have a LCI database without CPC categories (which are necessary for the double-counting check), you can provide a mapping between the products and activities in the LCI database and the CPC categories. The mapping should be provided in a dataframe with the following columns:
- `Name`: the (partial) name of the product or activity in the LCI database
- `CPC`: the number and name of the corresponding CPC category
- `Search type`: whether the `Name` entry is an exactly the name to look for, or is contained in the full name
- `Where`: whether the `Name` entry is meant for products or activities

In [17]:
mapping_product_to_CPC.head()

,Name,CPC,Search type,Where
0,amine-based silica,"35310: Organic surface active agents, except soap",equals,Product
1,biodiesel,35491: Biodiesel,contains,Product
2,biogas,"17200: Coal gas, water gas, producer gas and s...",equals,Product
3,"biomass, used as fuel",31230: Wood in chips or particles,equals,Product
4,"biomethane, from biogas upgrading, using amine...","12020: Natural gas, liquefied or in the gaseou...",equals,Product


An abbreviation scheme for the impact categories you aim to work with, to ease the readability in the ESM. The abbreviations should be provided in a dataframe with the following columns:
- `Impact_category`: the name of the impact category, expressed as a tuple following brightway convention
- `Unit`: (optional) the unit of the impact category
- `Abbrev`: the abbreviation of the impact category
- `AoP`: the area of protection of the impact category

In [18]:
impact_abbrev.head()

,Impact_category,Unit,Abbrev,AoP
0,"('IMPACT World+ Damage 2.0.1', 'Ecosystem qual...",PDF.m2.yr,CCEQL,EQ
1,"('IMPACT World+ Damage 2.0.1', 'Ecosystem qual...",PDF.m2.yr,CCEQS,EQ
2,"('IMPACT World+ Damage 2.0.1', 'Ecosystem qual...",PDF.m2.yr,CCEQLB,EQ
3,"('IMPACT World+ Damage 2.0.1', 'Ecosystem qual...",PDF.m2.yr,CCEQSB,EQ
4,"('IMPACT World+ Damage 2.0.1', 'Human health',...",DALY,CCHHL,HH


In case you want to remove some energy technologies from energy layers in the results LCI datasets to be added to the LCI database, you can provide a list of technologies to remove. The list should be provided in a dataframe with the following columns:
- `Layers`: name of the layer(s). A layer is basically an energy vector, which is an output for some energy technologies and an input for some others. 
- `Technologies`: the name of the energy technology to remove from the layer(s)
- `Comment`: (optional) a comment on the removal

In [19]:
technologies_to_remove_from_layers.head()

,Layers,Technologies,Comment
0,"['ELECTRICITY_EHV', 'ELECTRICITY_HV']","['TRAFO_HE','TRAFO_EH']",The high and extra high voltage electricity ar...
1,['ELECTRICITY_LV'],['STO_ELEC'],The storage technologies should be removed (pr...
2,"['NG_HP', 'NG_EHP']","['NG_EXP_EH', 'NG_EXP_EH_COGEN', 'NG_COMP_HE',...",The high and extra high pressure natural gas a...
3,"['H2_LP', 'H2_MP', 'H2_HP', 'H2_EHP']","['H2_COMP_HE', 'H2_COMP_MH', 'H2_COMP_LM', 'H2...",All pressure levels for hydrogen are merged in...
4,"['HEAT_HIGH_T', 'HEAT_LOW_T_DHN']",['HT_LT'],The high and low heat production at the DHN le...


If you want to reformat your end-use types (e.g., output layer) in order to better fit the LCI database, you can provide a list of new end-use types. The list should be provided in a dataframe with the following columns:
- `Name`: name of technologies for which the end-use type should be changed
- `Search type`: whether the `Name` entry is an exactly the name to look for (`equals`), is contained in the full name (`contains`), or is the beginning of the full name (`startswith`)
- `Old`: the current end-use type
- `New`: the new end-use type

In [20]:
new_end_use_types.head()

,Name,Search type,Old,New
0,BUS,startswith,MOB_PUBLIC,MOB_PUBLIC_BUS
1,SCHOOLBUS,startswith,MOB_PUBLIC,MOB_PUBLIC_SCHOOLBUS
2,COACH,startswith,MOB_PUBLIC,MOB_PUBLIC_COACH
3,TRAIN,startswith,MOB_PUBLIC,MOB_PUBLIC_TRAIN
4,CAR,startswith,MOB_PRIVATE,MOB_PRIVATE_CAR


If you are facing changes in names or reference products due to a new version of premise, and do not want to modify your mapping file, you should provide the dataframe with the following columns:
- `Product - old`: old reference product of the activity
- `Activity - old`: old name of the activity
- `Location - old`: old location of the activity
- `Product - new`: new reference product of the activity
- `Activity - new`: new name of the activity
- `Location - new`: new location of the activity

In [21]:
premise_changes.head()

,Product - old,Activity - old,Location - old,Product - new,Activity - new,Location - new
0,"hydrogen, gaseous, 25 bar","hydrogen production, auto-thermal reforming of...",RER,"hydrogen, gaseous, low pressure","hydrogen production, auto-thermal reforming",RER
1,"hydrogen, gaseous, 25 bar","hydrogen production, auto-thermal reforming of...",RER,"hydrogen, gaseous, low pressure","hydrogen production, auto-thermal reforming, w...",RER
2,"hydrogen, gaseous, 25 bar","hydrogen production, auto-thermal reforming, f...",CH,"hydrogen, gaseous, low pressure","hydrogen production, auto-thermal reforming, f...",RER
3,"hydrogen, gaseous, 25 bar","hydrogen production, auto-thermal reforming, f...",CH,"hydrogen, gaseous, low pressure","hydrogen production, auto-thermal reforming, f...",RER
4,"hydrogen, gaseous, 26 bar","hydrogen production, steam methane reforming, ...",CH,"hydrogen, gaseous, low pressure","hydrogen production, steam methane reforming, ...",RER


If you want to inject the results of your ESM back in the LCI database, you should provide the results in a dataframe with the following columns:
- `Name`: the name of the energy technology in the ESM
- `Production`: the annual production value of the energy technology in the ESM. All values should be provided with the same unit.

In [22]:
results_from_esm.head()

,Name,Production
0,PV,0.000000
1,TRAFO_ML,72698.861206
2,AN_DIG_SI,0.000000
3,GEOTHERMAL,0.000000
4,HYDRO_GAS_CHP,0.000000


## Create a new database with additional CPC categories (optional)
In case you are working with a LCI database without CPC categories, you can create a new database with the CPC categories. The function `create_new_database_with_CPC_categories` takes as input the database with missing CPC categories, the name of the new database, and a mapping between the products and activities in the LCI database and the CPC categories. It creates a new database with the CPC categories. This step can take a few minutes depending on the size of the database.

In [23]:
if name_premise_with_CPC_db not in bd.databases: # if the database with CPC categories does not exist, create it
    premise_db = load_extract_db(name_premise_db, create_pickle=True)
    create_new_database_with_CPC_categories(db=premise_db, new_db_name=name_premise_with_CPC_db)

## Operations on the mapping dataframe (optional)

In [24]:
premise_db_with_CPC = load_extract_db(name_premise_with_CPC_db, create_pickle=True)

### Relink the mapping dataframe with another database
In this example, we relink the current mapping, which is based on ecoinvent and some additional inventories, with a [premise](https://premise.readthedocs.io/en/latest/introduction.html) database. The function `create_complementary_database` takes as input the mapping dataframe, the database to link to, and the name of a newly created complementary database, in case some LCI datasets are missing in the LCI database we relink to. It returns a new mapping dataframe with the location column updated, and create the complementary database in the brightway project.

In [25]:
name_premise_comp_with_CPC_db = name_premise_with_CPC_db + f'_comp_{esm_location}'

In [26]:
# Testing if the mapping file is correctly linking the ESM with the LCI database
base_db = load_multiple_databases(list(mapping['Database'].unique()))
unlinked = test_mapping_file(mapping, base_db)

Getting activity data


100%|██████████| 939/939 [00:00<00:00, 56457.16it/s]


Adding exchange data to activities


100%|██████████| 30812/30812 [00:03<00:00, 8142.00it/s] 


Filling out exchange data


...

Adding exchange data to activities


100%|██████████| 30702/30702 [00:02<00:00, 14497.11it/s]


Filling out exchange data


100%|██████████| 935/935 [00:01<00:00, 484.32it/s]


Mapping successfully linked to the database


In [27]:
mapping_linked_to_premise = create_complementary_database(mapping, premise_db_with_CPC, name_premise_comp_with_CPC_db, premise_changes)

Getting activity data


100%|██████████| 939/939 [00:00<00:00, 117292.62it/s]


Adding exchange data to activities


100%|██████████| 30812/30812 [00:02<00:00, 14995.52it/s]


Filling out exchange data


...

Adding exchange data to activities


100%|██████████| 30702/30702 [00:01<00:00, 16392.84it/s]


Filling out exchange data


100%|██████████| 935/935 [00:16<00:00, 57.25it/s] 


The complementary database did not have to be created


In [28]:
if name_premise_db+f'_comp_{esm_location}' in bd.databases: # if the complementary database exists, load it and update it with CPC categories
    premise_comp_db = load_extract_db(name_premise_comp_with_CPC_db) # complementary database without CPC categories
    create_new_database_with_CPC_categories(db=premise_comp_db, new_db_name=name_premise_comp_with_CPC_db)
    premise_comp_db = load_extract_db(name_premise_comp_with_CPC_db) # complementary database with CPC categories
    main_db = premise_db_with_CPC + premise_comp_db
else:
    main_db = premise_db_with_CPC

In [29]:
mapping_linked_to_premise.head()

,Name,Type,Product,Activity,Location,Database
0,AFC,Construction,"fuel cell system, 1 kWe, proton exchange membr...","fuel cell system assembly, 1 kWe, proton excha...",GLO,ecoinvent_cutoff_3.9.1_image_SSP2-Base_2050+tr...
1,AFC,Operation,"electricity, from residential heating system","electricity, residential, by conversion of hyd...",CH,ecoinvent_cutoff_3.9.1_image_SSP2-Base_2050+tr...
2,ALKALINE_ELECTROLYSIS,Operation,"hydrogen, gaseous, 20 bar","hydrogen production, gaseous, 20 bar, from AEC...",CH,ecoinvent_cutoff_3.9.1_image_SSP2-Base_2050+tr...
3,ALKALINE_ELECTROLYSIS_PLANT,Construction,"electrolyzer, 1MWe, AEC, Balance of Plant","electrolyzer production, 1MWe, AEC, Balance of...",RER,ecoinvent_cutoff_3.9.1_image_SSP2-Base_2050+tr...
4,ALKALINE_ELECTROLYSIS_PLANT_DECOM,Construction,"used fuel cell balance of plant, 1MWe, AEC","treatment of fuel cell balance of plant, 1MWe,...",RER,ecoinvent_cutoff_3.9.1_image_SSP2-Base_2050+tr...


### Add or replace the location column based on a user-defined ranking
Based on a user-defined ranking, the location column of the mapping dataframe can be updated. The function `change_location_mapping_file` takes as input the mapping dataframe, the user-defined ranking, and the base database. It returns the mapping dataframe with the location column updated.

In [30]:
# create a concatenated database of all databases in the mapping dataframe (including background requirements, except biosphere databases)
base_db = load_multiple_databases(list(mapping_linked_to_premise.Database.unique()))

In [31]:
# Define the user-defined ranking
if esm_location == 'CA-QC':
    my_ranking = [
        'CA-QC', # Quebec
        'CA', # Canada
        'CAN', # Canada in IMAGE
        'CAZ', # Canada - Australia - New Zealand in REMIND
        'RNA', # North America
        'US', # United States
        'USA', # United States in REMIND and IMAGE
        'GLO', # Global average 
        'RoW', # Rest of the world
    ]
elif esm_location == 'CH':
    my_ranking = [
        'CH',
        'NEU',
        'EUR',
        'WEU',
        'RER', 
        'IAI Area, EU27 & EFTA',
        'GLO',
        'RoW'
    ]
else:
    my_ranking = [
        'GLO',
        'RoW',
    ]

In [32]:
# Update mapping dataframe with better locations
mapping_linked_to_premise = change_location_mapping_file(mapping_linked_to_premise, my_ranking, base_db, esm_location)

In [33]:
mapping_linked_to_premise.head()

,Name,Type,Product,Activity,Location,Database
0,AFC,Construction,"fuel cell system, 1 kWe, proton exchange membr...","fuel cell system assembly, 1 kWe, proton excha...",GLO,ecoinvent_cutoff_3.9.1_image_SSP2-Base_2050+tr...
1,AFC,Operation,"electricity, from residential heating system","electricity, residential, by conversion of hyd...",CH,ecoinvent_cutoff_3.9.1_image_SSP2-Base_2050+tr...
2,ALKALINE_ELECTROLYSIS,Operation,"hydrogen, gaseous, 20 bar","hydrogen production, gaseous, 20 bar, from AEC...",CH,ecoinvent_cutoff_3.9.1_image_SSP2-Base_2050+tr...
3,ALKALINE_ELECTROLYSIS_PLANT,Construction,"electrolyzer, 1MWe, AEC, Balance of Plant","electrolyzer production, 1MWe, AEC, Balance of...",RER,ecoinvent_cutoff_3.9.1_image_SSP2-Base_2050+tr...
4,ALKALINE_ELECTROLYSIS_PLANT_DECOM,Construction,"used fuel cell balance of plant, 1MWe, AEC","treatment of fuel cell balance of plant, 1MWe,...",RER,ecoinvent_cutoff_3.9.1_image_SSP2-Base_2050+tr...


In [34]:
# Save the relinked mapping file for later use
mapping_linked_to_premise.to_csv(f'../dev/energyscope_data/{esm_location}/mapping_{ecoinvent_version}_linked.csv', index=False)

## Double-counting removal

In [35]:
# Load the relinked mapping file to skip the previous steps
mapping_linked_to_premise = pd.read_csv(f'../dev/energyscope_data/{esm_location}/mapping_{ecoinvent_version}_linked.csv')

In [36]:
mapping_linked_to_premise_wo_flows = mapping_linked_to_premise[mapping_linked_to_premise['Type'] != 'Flow'] # remove flows from the mapping, only keep technologies and resources

In [37]:
# sufficient match within ecoinvent
if esm_location == 'CA-QC':
    accepted_locations = ['CA-QC', 'CA']  
elif esm_location == 'CH':
    accepted_locations = ['CH']  
else:
    accepted_locations = ['GLO', 'RoW']  

In [38]:
base_db = load_multiple_databases(list(mapping_linked_to_premise_wo_flows.Database.unique()))
if regionalize_foregrounds & regionalized_database:
    regionalized_biosphere_db = load_extract_db('biosphere3_regionalized_flows')
else:
    regionalized_biosphere_db = None

Getting activity data


100%|██████████| 61899/61899 [00:02<00:00, 23955.35it/s] 


Adding exchange data to activities


0it [00:00, ?it/s]


Filling out exchange data


100%|██████████| 61899/61899 [00:00<00:00, 3423076.32it/s]


In [39]:
# Create the ESM database in your brightway project
mapping_linked_to_premise_new_code = create_esm_database(
    mapping=mapping_linked_to_premise_wo_flows,
    model=model,
    unit_conversion=unit_conversion,
    tech_specifics=technology_specifics,
    technology_compositions=technology_compositions,
    mapping_esm_flows_to_CPC_cat=mapping_esm_flows_to_CPC,
    #efficiency=efficiency,
    main_database=base_db,
    esm_db_name=new_db_name,
    regionalize_foregrounds=regionalize_foregrounds,
    accepted_locations=accepted_locations,
    target_region=esm_location,
    locations_ranking=my_ranking,
    results_path_file=f'results/energyscope_{esm_location}/{esm_year}/',
    regionalized_database=regionalized_database,
    regionalized_biosphere_db=regionalized_biosphere_db,
)

No location found in your ranking for waste polystyrene - market group for waste polystyrene - ecoinvent_cutoff_3.9.1_image_SSP2-Base_2050+truck_carculator regionalized_with_CPC
Have to keep the initial location: Europe without Switzerland
No location found in your ranking for waste polyvinylchloride - market group for waste polyvinylchloride - ecoinvent_cutoff_3.9.1_image_SSP2-Base_2050+truck_carculator regionalized_with_CPC
Have to keep the initial location: Europe without Switzerland
No location found in your ranking for waste polystyrene - market group for waste polystyrene - ecoinvent_cutoff_3.9.1_image_SSP2-Base_2050+truck_carculator regionalized_with_CPC
Have to keep the initial location: Europe without Switzerland
No location found in your ranking for waste polyvinylchloride - market group for waste polyvinylchloride - ecoinvent_cutoff_3.9.1_image_SSP2-Base_2050+truck_carculator regionalized_with_CPC
Have to keep the initial location: Europe without Switzerland
No location foun

Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:01


Title: Writing activities to SQLite3 database:
  Started: 07/25/2024 16:35:34
  Finished: 07/25/2024 16:35:36
  Total time elapsed: 00:00:01
  CPU %: 41.70
  Memory %: 6.47


In [40]:
# Save the relinked mapping file for later use
mapping_linked_to_premise_new_code.to_csv(f'../dev/energyscope_data/{esm_location}/mapping_{ecoinvent_version}_new_code.csv', index=False)

## Computing the LCA metrics 

In [41]:
# To skip the previous steps (if the relinked mapping file has already been created)
# mapping_linked_to_premise_new_code = pd.read_csv(f'../dev/energyscope_data/{esm_location}/mapping_{ecoinvent_version}_new_code.csv')

In [42]:
esm_db = load_extract_db(new_db_name)

Getting activity data


100%|██████████| 1143/1143 [00:00<00:00, 67109.33it/s]


Adding exchange data to activities


100%|██████████| 32396/32396 [00:00<00:00, 38883.98it/s]


Filling out exchange data


100%|██████████| 1143/1143 [00:01<00:00, 671.07it/s]


In [43]:
# Compute the impact scores
R_long = compute_impact_scores(
    esm_db=esm_db,
    mapping=mapping_linked_to_premise_new_code,
    technology_compositions=technology_compositions,
    methods=lcia_methods,
    unit_conversion=unit_conversion,
    lifetime=lifetime,
)

In [44]:
R_long.Value *= 1e6 # from FU / kW(h)-pkm-tkm to FU / GW(h)-Mpkm-Mtkm

In [45]:
R_long.head()

,Impact_category,New_code,Value,Name,Type
0,"(IMPACT World+ Damage 2.0.1_regionalized, Ecos...",bfa2adtz5ne01vt7mkzqkdzad68vi00u,3.242398e+09,AFC,Construction
1,"(IMPACT World+ Damage 2.0.1_regionalized, Ecos...",bfa2adtz5ne01vt7mkzqkdzad68vi00u,1.067099e+09,AFC,Construction
2,"(IMPACT World+ Damage 2.0.1_regionalized, Huma...",bfa2adtz5ne01vt7mkzqkdzad68vi00u,1.475618e+04,AFC,Construction
3,"(IMPACT World+ Damage 2.0.1_regionalized, Huma...",bfa2adtz5ne01vt7mkzqkdzad68vi00u,4.931528e+03,AFC,Construction
4,"(IMPACT World+ Midpoint 2.0.1_regionalized, Mi...",bfa2adtz5ne01vt7mkzqkdzad68vi00u,5.629137e+09,AFC,Construction


In [46]:
R_long.to_csv(f'results/energyscope_{esm_location}/{esm_year}/impact_scores.csv', index=False)

## Convert the results in an AMPL format

In [47]:
# To skip the previous steps
R_long = pd.read_csv(f'results/energyscope_{esm_location}/{esm_year}/impact_scores.csv')

In [48]:
refactor = 1e-2
lcia_method = 'IMPACT World+ Damage 2.0.1_regionalized - Total only'

### Create the .dat file

In [49]:
normalize_lca_metrics(
    R=R_long,
    mip_gap=1e-6,
    refactor=refactor,
    lcia_method=lcia_method,
    impact_abbrev=impact_abbrev,
    path=f'results/energyscope_{esm_location}/{esm_year}/',
)

### Create the .mod file

In [50]:
gen_lcia_obj(
    lcia_method=lcia_method,
    refactor=refactor,
    impact_abbrev=impact_abbrev,
    path=f'results/energyscope_{esm_location}/{esm_year}/',
)

## Integrate the ESM results back in the LCI database

In [51]:
base_db = load_multiple_databases(list(mapping_linked_to_premise_wo_flows.Database.unique())) # reset the database

In [52]:
create_new_database_with_esm_results(
    mapping=mapping_linked_to_premise,
    model=model,
    esm_location=esm_location,
    esm_results=results_from_esm,
    unit_conversion=unit_conversion,
    db=base_db,
    locations_ranking=my_ranking,
    accepted_locations=accepted_locations,
    new_db_name=f'ecoinvent_cutoff_{ecoinvent_version}_{premise_iam}_{premise_ssp_rcp}_{esm_year}_with_ESM_results_for_{esm_location}',
    technology_compositions=technology_compositions,
    tech_specifics=technology_specifics,
    mapping_esm_flows_to_CPC_cat=mapping_esm_flows_to_CPC,
    tech_to_remove_layers=technologies_to_remove_from_layers,
    new_end_use_types=new_end_use_types,
    regionalized_database=regionalized_database,
    regionalized_biosphere_db=regionalized_biosphere_db,
)

No location found in your ranking for hard coal - hard coal, import from RNA - ecoinvent_cutoff_3.9.1_image_SSP2-Base_2050+truck_carculator regionalized_with_CPC
Have to keep the initial location: IN
No location found in your ranking for hard coal - hard coal, import from ZA - ecoinvent_cutoff_3.9.1_image_SSP2-Base_2050+truck_carculator regionalized_with_CPC
Have to keep the initial location: IN
No location found in your ranking for hard coal - hard coal, import from RU - ecoinvent_cutoff_3.9.1_image_SSP2-Base_2050+truck_carculator regionalized_with_CPC
Have to keep the initial location: IN
The technology DHN_RENOVATION is not in the mapping file. It cannot be considered in the result LCI dataset.
No location found in your ranking for kerosene - kerosene, import from RoW - ecoinvent_cutoff_3.9.1_image_SSP2-Base_2050+truck_carculator regionalized_with_CPC
Have to keep the initial location: ZA
No location found in your ranking for natural gas, high pressure - evaporation of natural gas -

Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:01:26


Title: Writing activities to SQLite3 database:
  Started: 07/25/2024 17:13:22
  Finished: 07/25/2024 17:14:48
  Total time elapsed: 00:01:26
  CPU %: 58.60
  Memory %: 23.32
